In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv, global_mean_pool
from torch_geometric.data import Data
#from graph_builder import GraphBuilder  # <-- External builder
import pandas as pd
from torch.nn import Linear, ReLU, Sequential
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_add_pool, GraphNorm
from sklearn.model_selection import train_test_split
import ast
from torch_geometric.utils import degree

In [16]:
from GraphBuilder_single_edge import GraphBuilder
from save_model_results import save_model_architecture, append_evaluation_results, evaluate_model

## Model architecture

In [3]:
class SimpleGNN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels,dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.norm1 = GraphNorm(hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.norm2 = GraphNorm(hidden_channels,hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.norm3 = GraphNorm(hidden_channels)
        self.lin = torch.nn.Linear(hidden_channels, 2)
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        batch = data.batch # For multiple graphs in a batch
        x = F.relu(self.norm1(self.conv1(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.norm2(self.conv2(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.norm3(self.conv3(x, edge_index)))
        x = self.dropout(x)
        x = global_mean_pool(x, batch)
        return self.lin(x)

In [4]:
# Create the model object
model = SimpleGNN(in_channels=4, hidden_channels=32,dropout=0)

In [5]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, weight_decay=5e-4)

In [6]:
# Generate text file with model architecture
model_results_path = save_model_architecture(model)

## 8-loop 

In [11]:
loop = 8

### Data preparation

First we read the edges and coefficients of the csv files and save them in lists.

In [12]:
# Create the edge and y lists from the csv files\
edges=[]
y=[]
for i in range(loop,loop+1):
    filename = f'../Graph_Edge_Data/den_graph_data_{i}.csv'
    df = pd.read_csv(filename)
    edges += df['EDGES'].tolist()
    y += df['COEFFICIENTS'].tolist()
edges = [ast.literal_eval(e) for e in edges]    

In [13]:
# Import the function to add eigenvector features
from torch_geometric.transforms import AddLaplacianEigenvectorPE
eigen_vec= AddLaplacianEigenvectorPE(k=3,attr_name=None)

We need to now translate the edges into dataset forms for training and testing.

In [17]:
# Define the data object through GraphBuilder, then add the eigenvector features
data=[GraphBuilder(solid_edges=x,coeff=y0).build() for x,y0 in zip(edges,y)]
data = [eigen_vec(d) for d in data]

NameError: name 'torch' is not defined

In [ ]:
# Split train and test data
train_data, test_data = train_test_split(data, test_size=0.1, random_state=43)

In [ ]:
# Load the data into DataLoader
train_loader = DataLoader(train_data, batch_size=5, shuffle=True)
test_loader = DataLoader(test_data, batch_size=5, shuffle=False)

### Model training

We are interested in graph classification of 0 and 1. We add two graph convolutional layers, making sure that the message passing is extended to two neighbours, and then add graph pooling to average over the whole graph.

In [ ]:
def train_model(model, train_loader, test_loader, optimizer, criterion, device, n_epochs=200):
    accuracy_list = []
    loss_list = []
    patience_counter = 0
    patience = 3

    model.to(device)

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0

        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            out = model(batch)             # out = model(batch) handles batch.x, batch.edge_index, etc.
            loss = criterion(out, batch.coeff) # Use batch.y (or batch.coeff if that's what your dataset uses)

            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        loss_list.append(total_loss)

        # Validation
        model.eval()
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)
                out = model(batch)
                _, predicted = torch.max(out, 1)
                correct += (predicted == batch.coeff).sum().item()
                total += batch.num_graphs  # graph-level classification

        accuracy = correct / total
        accuracy_list.append(accuracy)

        print(f"Epoch {epoch+1}: Loss={total_loss:.4f}, Accuracy={accuracy:.4f}")

        if epoch>100: 
            if loss_list[epoch-10]-loss_list[epoch] < 0.01:
                patience_counter += 1
                if patience_counter >= patience:
                    print("Early stopping") 
                    break 
         

                


In [ ]:
train_model(model,train_loader,test_loader,optimizer,device='cpu',criterion=criterion)

Epoch 1: Loss=99.2483, Accuracy=0.7014
Epoch 2: Loss=101.8600, Accuracy=0.7083
Epoch 3: Loss=97.2561, Accuracy=0.7639


KeyboardInterrupt: 

### Model evaluation

In [ ]:
evaluation =evaluate_model(model, test_loader, device='cpu')

{'Accuracy': 0.6944444444444444,
 'Precision': array([0.6875, 0.7   ]),
 'Recall': array([0.64705882, 0.73684211]),
 'F1 Score': array([0.66666667, 0.71794872]),
 'Confusion Matrix': array([[44, 24],
        [20, 56]])}

In [ ]:
append_evaluation_results(model_results_path, evaluation, loop = loop )